In [1]:
// In python use: from pyspark.sql.functions import broadcast, split, lit
import org.apache.spark.sql.functions.{broadcast, split, lit}


val matchesBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/matches.csv")
val matchDetailsBucketed =  spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/match_details.csv")

Intitializing Scala interpreter ...

Spark Web UI available at http://f9c8cdc0638d:4041
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1733499274162)
SparkSession available as 'spark'


import org.apache.spark.sql.functions.{broadcast, split, lit}
matchesBucketed: org.apache.spark.sql.DataFrame = [match_id: string, mapid: string ... 8 more fields]
matchDetailsBucketed: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 34 more fields]


In [2]:
spark.sql("""DROP TABLE IF EXISTS bootcamp.matches_bucketed""")

res0: org.apache.spark.sql.DataFrame = []


In [3]:
val bucketedDDL = """CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
                         match_id STRING,
                         is_team_game BOOLEAN,
                         playlist_id STRING,
                         completion_date TIMESTAMP
                     )
                     USING iceberg;
                     """
spark.sql(bucketedDDL)

bucketedDDL: String =
"CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
                         match_id STRING,
                         is_team_game BOOLEAN,
                         playlist_id STRING,
                         completion_date TIMESTAMP
                     )
                     USING iceberg;
                     "
res1: org.apache.spark.sql.DataFrame = []


In [4]:
matchesBucketed.select($"match_id", $"is_team_game", $"playlist_id", $"completion_date")
    .write.mode("append")
    .saveAsTable("bootcamp.matches_bucketed")

In [5]:
// After creating table
spark.sql("SELECT * FROM bootcamp.matches_bucketed LIMIT 5").show()

// After writing data
spark.sql("SELECT COUNT(*) FROM bootcamp.matches_bucketed").show()

+--------------------+------------+--------------------+-------------------+
|            match_id|is_team_game|         playlist_id|    completion_date|
+--------------------+------------+--------------------+-------------------+
|11de1a94-8d07-416...|        true|f72e0ef0-7c4a-430...|2016-02-22 00:00:00|
|d3643e71-3e51-43e...|       false|d0766624-dbd7-453...|2016-02-14 00:00:00|
|d78d2aae-36e4-48a...|        true|f72e0ef0-7c4a-430...|2016-03-24 00:00:00|
|b440069e-ec5f-4f5...|        true|f72e0ef0-7c4a-430...|2015-12-23 00:00:00|
|1dd475fc-ee6b-4e1...|        true|0e39ead4-383b-445...|2016-04-07 00:00:00|
+--------------------+------------+--------------------+-------------------+

+--------+
|count(1)|
+--------+
|   24025|
+--------+



In [ ]:
val bucketedDetailsDDL = """
CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
     match_id STRING,
     player_gamertag STRING,
     player_total_kills INTEGER,
     player_total_deaths INTEGER
 )
 USING iceberg
 PARTITIONED BY (bucket(16, match_id));
 """
 spark.sql(bucketedDetailsDDL)

matchDetailsBucketed.select(
    $"match_id", $"player_gamertag", $"player_total_kills", $"player_total_deaths")
    .write.mode("append")
  .bucketBy(16, "match_id").saveAsTable("bootcamp.match_details_bucketed")

In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

matchesBucketed.createOrReplaceTempView("matches")
matchDetailsBucketed.createOrReplaceTempView("match_details")

spark.sql("""
    SELECT * FROM bootcamp.match_details_bucketed mdb JOIN bootcamp.matches_bucketed md 
    ON mdb.match_id = md.match_id
    AND md.completion_date = DATE('2016-01-01')    
""").explain()

In [ ]:



spark.sql("""
    SELECT * FROM match_details mdb JOIN matches md ON mdb.match_id = md.match_id
        
""").explain()

 spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "1000000000000")

 val broadcastFromThreshold = matches.as("m").join(matchDetails.as("md"), $"m.match_id" === $"md.match_id")
   .select($"m.completion_date", $"md.player_gamertag",  $"md.player_total_kills")
   .take(5)

 val explicitBroadcast = matches.as("m").join(broadcast(matchDetails).as("md"), $"m.match_id" === $"md.match_id")
   .select($"md.*", split($"completion_date", " ").getItem(0).as("ds"))

 val bucketedValues = matchDetailsBucketed.as("mdb").join(matchesBucketed.as("mb"), $"mb.match_id" === $"mdb.match_id").explain()
 .take(5)

 val values = matchDetailsBucketed.as("m").join(matchesBucketed.as("md"), $"m.match_id" === $"md.match_id").explain()

 explicitBroadcast.write.mode("overwrite").insertInto("match_details_bucketed")

 matches.withColumn("ds", split($"completion_date", " ").getItem(0)).write.mode("overwrite").insertInto("matches_bucketed")

 spark.sql(bucketedSQL)

